# Data

## Imports

In [1]:
import ROOT
import numpy as np
import fnmatch
from os import listdir

RDataFrame = ROOT.RDF.Experimental.Distributed.Spark.RDataFrame
RunGraphs = ROOT.RDF.Experimental.Distributed.RunGraphs
initialize = ROOT.RDF.Experimental.Distributed.initialize

Welcome to JupyROOT 6.27/01


## Init strings

Choose the function and the parameter that's used for creating the weights- Function needs to be known by ROOT.

In [2]:
fitFunction = "chebyshev8"
fitParameter = "Jet_nConstituents"

## Initialize RDFs

### Read

Collect the .root files to a single RDF

In [3]:
# This line finds all files in the directory that end with .root
files1 = list(fnmatch.filter(listdir(
    "../../../data/20UL18JMENano_106X_upgrade2018_realistic_v16_L1v1-v1/30000"), "*.root"))
files2 = list(fnmatch.filter(
    listdir("../../../data/UL2018_MiniAODv2_JMENanoAODv9-v1/40000"), "*.root"))

# Load the files and create the RDataFrame
chain1 = ROOT.TChain("Events")
for file in files1:
    chain1.Add("root://eosuser.cern.ch//eos/user/n/ntoikka/data/20UL18JMENano_106X_upgrade2018_realistic_v16_L1v1-v1/30000/"+file)

entr1 = chain1.GetEntries()
print(f"Event count (MC): {entr1}")
print(f"File count (MC): {len(files1)}")
print()

for file in files2:
    chain1.Add(
        "root://eosuser.cern.ch//eos/user/n/ntoikka/data/UL2018_MiniAODv2_JMENanoAODv9-v1/40000/"+file)
    
print(f"Event count (DT): {chain1.GetEntries() - entr1}")
print(f"File count (DT): {len(files2)}")

df = RDataFrame(chain1, sparkcontext=sc, npartitions=128)

df._headnode.backend.distribute_headers("../macros/Nhelpers.hpp")
df._headnode.backend.distribute_headers("../macros/tdrstyle_mod15.hpp")

Event count (MC): 19928000
File count (MC): 39

Event count (DT): 78255208
File count (DT): 128


Warning in <TClass::Init>: no dictionary for class edm::Hash<1> is available
Warning in <TClass::Init>: no dictionary for class edm::ProcessHistory is available
Warning in <TClass::Init>: no dictionary for class edm::ProcessConfiguration is available
Warning in <TClass::Init>: no dictionary for class edm::ParameterSetBlob is available
Warning in <TClass::Init>: no dictionary for class pair<edm::Hash<1>,edm::ParameterSetBlob> is available


### MC-DT separation

Define new column that tells whether the event came from a DT file or an MC file.

In [4]:
DTvMC_filt = "(false"

for filename in files1:
    DTvMC_filt += f' || (rdfsampleinfo_.Contains("{filename}"))'

DTvMC_filt += ") ? 1 : 2"

# fileN column contains 1 for MC and 2 for DT
df1 = df.DefinePerSample("fileN", DTvMC_filt)

## Transform RDFs

Apply filters and defines to the collective RDF. After transformation separate DT and MC to separate RDFs.

Filter string to match based on trigger

In [5]:
hlt = [40, 60, 80, 140, 200, 260, 320, 400, 450, 500]
offline = [64, 84, 114, 196, 272, 330, 395, 468, 548, 686]
hlt_filt = "false "

for hlt_temp, reco in zip(hlt, offline):
    hlt_filt += f" || ((pTavg > {reco}) && (HLT_PFJet{hlt_temp}))"

Transformation and separation

In [6]:
df2 = df1.Filter("(Jet_pt.size() >= 2)") \
    .Define("pTavg", "(Jet_pt[0] + Jet_pt[1]) * 0.5") \
    .Filter("(Jet_pt.size() == 2) || ((Jet_pt[2] < 15) || (Jet_pt[2] < 0.3*pTavg))") \
    .Filter("ROOT::VecOps::DeltaPhi(Jet_phi[0], Jet_phi[1]) > 2.8") \
    .Filter("(Jet_pt[0] > 15) && (Jet_pt[1] > 15)") \
    .Filter("(abs(Jet_eta[0]) < 1.3) && (abs(Jet_eta[1]) < 1.3)") \
    .Filter(hlt_filt) \
    .Define("rIdx", "rand_oneZero()") \
    .Redefine(fitParameter, f"Take({fitParameter}, rIdx)") \
    .Define("pTtag", "Take(Jet_pt, rIdx)") \
    .Filter("pTtag[0] < 1700 and pTtag[0] > 64")

df_MC = df2.Filter("fileN == 1")
df_DT = df2.Filter("fileN == 2")

## Hists

Create 2D histograms for finding the weights

Parameters

In [7]:
bins = np.array((1, 5, 6, 8, 10, 12, 15, 18, 21, 24, 28, 32, 37, 43, 49, 56, 64, 74, 84, 97, 114, 133,
                 153, 174, 196, 220, 245, 272, 300, 330, 362, 395, 430, 468, 507, 548, 592, 638, 686, 737,
                 790, 846, 905, 967, 1032, 1101, 1172, 1248, 1327, 1410, 1497, 1588, 1684, 1784, 1890, 2000,
                 2116, 2238, 2366, 2500, 2640, 2787, 2941, 3103, 3273, 3450, 3637, 3832, 4037, 4252, 4477, 4713,
                 4961, 5220, 5492, 5777, 6076, 6389, 6717, 7000), dtype=np.double)
cnt = len(bins)-1

yMin = df_DT.Min(fitParameter)
yMax = df_DT.Max(fitParameter)

RunGraphs([yMin, yMax])

yMin = yMin.GetValue()
yMax = yMax.GetValue()

yBins = int((yMax - yMin))

Histograms

In [8]:
DT_Hist = df_DT.Histo2D(("dtHist", "dtHist", cnt, bins,
                         yBins, yMin, yMax), "pTtag", fitParameter)
MC_recoHist = df_MC.Histo2D(
    ("mcRecoHist", "mcRecoHist", cnt, bins, yBins, yMin, yMax), "pTtag", fitParameter)

RunGraphs([DT_Hist, MC_recoHist])

DT_hist = DT_Hist.GetValue()
MC_recoHist = MC_recoHist.GetValue()

Save the results

In [9]:
myfile = ROOT.TFile( 'histos.root', 'RECREATE' )
DT_Hist.Write()
MC_recoHist.Write()
myfile.Close()

## Fits and weights

### C++ code for workers

Functions and variables for calculating and storing the weighting functions. Should be transferred to .hpp for better performance

In [10]:
def init():
    weight_code = """ 
    #ifndef MYFUN
    #define MYFUN
    
    #include <map>
    #include <string>
    
    // Dictionary for pT (keys) and function (values) pairs
    std::map<Float_t, TF1 *> fitDic; 
    
    // Keys
    int bin_count = 79;
    std::vector<Float_t> bins{1, 5, 6, 8, 10, 12, 15, 18, 21, 24, 28, 32, 37, 43, 49, 56, 64, 74, 84, 97, 114, 133, 153, 174, 196, 220, 245, 272, 300, 330, 362, 395, 430, 468, 507, 548, 592, 638, 686, 737,790, 846, 905, 967, 1032, 1101, 1172, 1248, 1327, 1410, 1497, 1588, 1684, 1784, 1890, 2000,2116, 2238, 2366, 2500, 2640, 2787, 2941, 3103, 3273, 3450, 3637, 3832, 4037, 4252, 4477, 4713,4961, 5220, 5492, 5777, 6076, 6389, 6717, 7000};  

    // Find the last non-empty bin in a histogram
    Float_t largestBin(TH1D* histogram) {
        int idx, Ncells = histogram->GetNbinsX();
        Double_t binContent;

        for (int i=Ncells; i>=1; i--) {
            binContent = histogram->GetBinContent(i);
            if (binContent != 0.0) {
                idx = i;
                break;
            }
        }

        return histogram->GetXaxis()->GetBinLowEdge(idx+1);
    }
    
    // Find the first non-empty bin in a histogram
    Float_t smallestBin(TH1D* histogram) {
        int idx, Ncells = histogram->GetNbinsX();
        Double_t binContent;

        for (int i=1; i<=Ncells; i++) {
            binContent = histogram->GetBinContent(i);
            if (binContent != 0.0) {
                idx = i;
                break;
            }
        }

        return histogram->GetXaxis()->GetBinLowEdge(idx);
    }    
    
    // Fill the dictionary with the fitted functions
    void initDic(std::string fitName) {
        // Histograms for creating the functions with fitting
        TFile *f = TFile::Open("root://eosuser.cern.ch//eos/user/n/ntoikka/SWAN_projects/CERN_summer2022/weighting/histos.root");
        auto dtHist = (TH2D*)f->Get("dtHist");
        auto mcRecoHist = (TH2D*)f->Get("mcRecoHist");
    
        // Loop through the bins
        for(int i=0; i<bin_count; i++) {
            std::string dtName = "dtProj" + std::to_string(i);
            std::string mcName = "mcProj" + std::to_string(i);
            auto h1 = mcRecoHist->ProjectionY(mcName.c_str(), i+1, i+1);
            auto h2 = dtHist->ProjectionY(dtName.c_str(), i+1, i+1);
            double_t dtMin = smallestBin(h1);
            double_t dtMax = largestBin(h1);


            // Check that the projections aren't empty
            // If not, scale and create the fit
            if ((h1->Integral() != 0.0) && (h2->Integral() != 0.0)) {
                h1->Scale(1/(h1->Integral()));
                h2->Scale(1/(h2->Integral()));
                
                h2->Divide(h1);
                h2->Fit(fitName.c_str(), "Q", "", dtMin, dtMax);
                fitDic[bins[i]] = h2->GetFunction(fitName.c_str());
            } 
            else {
                std::string fName = "zero" + std::to_string(i);
                fitDic[bins[i]] = new TF1(fName.c_str(), "0*x", 0.0, 100.0);
            }
        }
    }
    
    // Use the dictionary to evaluate Nval according to a given pT and its corresponding function
    Float_t create_weight(Float_t pT, int Nval) {
        int idx;
        Float_t result;
        
        // Find the correct key
        for (int i=0; i < bin_count; i++) {
            if((pT >= bins[i]) && (pT < bins[i+1])) {
                idx=i;
                break;
            }
        }
        result = fitDic[bins[idx]]->Eval(Nval);
        if (result > 2.0) {
            result = -1.0;
        }
        return result;
    }
      
    #endif
    """

    ROOT.gInterpreter.Declare(weight_code)
    ROOT.initDic(fitFunction)


initialize(init)

Info in <TCanvas::MakeDefCanvas>:  created default TCanvas with name c1


Save the functions for visual checking. Note currently hard coded range!

In [11]:
%jsroot on
canv = ROOT.TCanvas("canv", "canv", 800,600)
canv.Divide(6,9)

empty = False

late = ROOT.TLatex()
late.SetNDC()

for i in range(54):
    c = canv.cd(i+1)
    dtName = "dtProj" + str(i)
    mcName = "mcProj" + str(i)
    h1 = MC_recoHist.ProjectionY(mcName, i+1, i+1)
    h2 = DT_Hist.ProjectionY(dtName, i+1, i+1)
    dtMin = ROOT.smallestBin(h1)
    dtMax = ROOT.largestBin(h1)

    if ((h1.Integral() != 0.0) and (h2.Integral() != 0.0)):
        h1.Scale(1/(h1.Integral()))
        h2.Scale(1/(h2.Integral()))
        h2.Divide(h1)
        h2.Draw()
        h2.Fit(fitFunction, "Q", "", dtMin, dtMax)
        
        late.DrawLatex(.575, .6, f'Min: {dtMin}')
        late.DrawLatex(.575, .55, f'Max: {dtMax}')
    else:
        empty = True

canv.Print(f"./plots/fits_{fitParameter}_{fitFunction}.pdf")

Info in <TCanvas::Print>: pdf file ./plots/fits_Jet_nConstituents_chebyshev8.pdf has been created


### Weights

Add a weights column to the MC RDF

In [12]:
df_MC1 = df_MC.Define("weights", f"create_weight(pTtag[0], {fitParameter}[0])")

# Analysis

## Distribution of weights

In [13]:
%jsroot on
h = df_MC1.Histo1D(('myhist', 'myhist', 100, 0, 2), "weights")
c = ROOT.TCanvas()
h.Draw()
c.Draw()

## Weighted vs non-weighted vs real data

### Direct comparsion

In [23]:
h1 = df_DT.Profile1D(('dt', 'dt', cnt, bins), "pTtag", fitParameter)
h2 = df_MC1.Profile1D(('w', 'w', cnt, bins), "pTtag", fitParameter, "weights")
h3 = df_MC1.Profile1D(('nw', 'nw', cnt, bins), "pTtag", fitParameter)

RunGraphs([h1, h2, h3])

h11 = h1.GetValue()
h21 = h2.GetValue()
h31 = h3.GetValue()

In [24]:
c1 = ROOT.TCanvas()
h11.GetYaxis().SetRangeUser(10, 50)
h11.GetXaxis().SetRangeUser(0, 1800)
h11.Draw("same plc pmc")
h21.Draw("same plc pmc")
h31.Draw("same plc pmc")
c1.Draw()
c1.Print(f"./plots/direct_{fitParameter}_{fitFunction}.pdf")

Info in <TCanvas::Print>: pdf file ./plots/direct_Jet_nConstituents_chebyshev8.pdf has been created


### Ratio comparison

In [25]:
h21.Divide(h11)
h31.Divide(h11)

True

In [26]:
c1 = ROOT.TCanvas()

h21.GetYaxis().SetRangeUser(0.98, 1.02)
h21.GetXaxis().SetRangeUser(55, 1800)
h21.Draw("same plc pmc hist p")
h31.Draw("same plc pmc hist p")
c1.SetLogx()
c1.Draw()
c1.Print(f"./plots/ratio_{fitParameter}_{fitFunction}.pdf")

Info in <TCanvas::Print>: pdf file ./plots/ratio_Jet_nConstituents_chebyshev8.pdf has been created


### Effect on pT response?

# Old stuff

In [18]:
def DeclareToCpp(**kwargs):
    for k, v in kwargs.items():
        ROOT.gInterpreter.Declare(f"namespace PyVars {{ auto &{k} = *reinterpret_cast<{type(v).__cpp_name__}*>({ROOT.addressof(v)}); }}")

In [19]:
# %jsroot on
# h1 = MC_recoHist.GetValue().ProjectionY(f"_reyuco", 18, 18)
# h1.Scale(1/h1.Integral())
# h2 = DT_Hist.GetValue().ProjectionY(f"_DT", 18, 18)
# h2.Scale(1/h2.Integral())
# # h3 = MC_genHist.GetValue().ProjectionY(f"_reco", 18, 18)
# h4 = h1.Divide(h2)

# c = ROOT.TCanvas("", "", 600, 600)
# c.Divide(2, 1)
# c.cd(1)
# # prof = df_MC1.Profile1D(("name", "title", 100, 0, 100), "GenJet_nConstituents", "Jet_nConstituents").Draw()
# # prof.Add(h1, h3)
# f = h1.Fit("chebyshev4", "S", xmin=5, xmax=55)
# h1.Draw()
# c.cd(2)
# h2.Draw("SAME PLC PMC")
# h1.Draw("SAME PLC PMC")
# c.Draw()

In [20]:
# def tdrAxes(hist, y_low, y_up):
#     hist.GetYaxis().SetRangeUser(y_low, y_up)
#     hist.GetXaxis().SetRangeUser(15, bins[-1])
#     hist.GetXaxis().SetMoreLogLabels()
#     hist.GetXaxis().SetNoExponent()


# canv = ROOT.TCanvas("canv", "canv", 800, 600)

# dim = int(np.floor(np.sqrt(cnt)))
# canv.Divide(dim, dim)
# late = ROOT.TLatex()
# late.SetNDC()

# skip = 7

# for i in range(dim*dim):
#     c = canv.cd(i+1)

#     h1 = MC_recoHist.GetValue().ProjectionY(
#         f"{i+skip}_reco", i+skip, i+skip).Draw("SAME PLC PMC")
#     h2 = MC_genHist.GetValue().ProjectionY(
#         f"{i+skip}_gen", i+skip, i+skip).Draw("SAME PLC PMC")
#     h3 = DT_Hist.GetValue().ProjectionY(
#         f"{i+skip}_DT", i+skip, i+skip).Draw("SAME PLC PMC")

#     tdrAxes(DT_Hist.GetValue(), 0, 100)

#     late.DrawLatex(.575, .7,
#                    f'{DT_Hist.GetXaxis().GetBinLowEdge(i+skip)} <= pT < {DT_Hist.GetXaxis().GetBinUpEdge(i+skip)}')

# # canv.Print("plots.pdf")
# canv.Draw()